In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import hdbscan
from itertools import product
from time import time
import umap
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import hdbscan
from itertools import product
from time import time
import umap
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# ================== 1. 读取数据 ==================
with open("../data/fashion/handled/pca64_itm_emb_np.pkl", "rb") as f:  # ✅ 改成 fashion
    item_emb = pickle.load(f)  # shape: [n_items, dim]
X = np.asarray(item_emb)
X = StandardScaler().fit_transform(X)

print(f"数据形状: {X.shape}")

# ================== 2. 参数网格 ==================
umap_grid = {
    "n_neighbors": [10, 20, 50],
    "min_dist": [0.0, 0.1],
    "n_components": [4, 8, 16],
    "metric": ["euclidean"]
}

hdb_grid = {
    "min_cluster_size": [20, 50, 100],
    "min_samples": [5, 10, 20, 50],
    "cluster_selection_epsilon": [0.0, 0.1],
    "metric": ["euclidean"]
}

umap_combos = list(product(*umap_grid.values()))
hdb_combos = list(product(*hdb_grid.values()))
total = len(umap_combos) * len(hdb_combos)

rows = []
idx = 0

# ================== 3. 遍历组合 ==================
for u_params in umap_combos:
    nn, md, nc, met_r = u_params

    reducer = umap.UMAP(
        n_neighbors=nn, min_dist=md, n_components=nc,
        metric=met_r, random_state=42
    )
    Z = reducer.fit_transform(X)   # 🔹 UMAP 降维

    for h_params in hdb_combos:
        mcs, ms, eps, met_c = h_params
        idx += 1
        t0 = time()

        clusterer = hdbscan.HDBSCAN(
            min_cluster_size=mcs, min_samples=ms,
            cluster_selection_epsilon=eps, metric=met_c
        )
        labels = clusterer.fit_predict(Z)

        # ---- 评估指标 ----
        noise_ratio = float(np.mean(labels == -1))
        mask = labels != -1
        if mask.sum() > 1 and len(np.unique(labels[mask])) >= 2:
            sil = float(silhouette_score(Z[mask], labels[mask]))
        else:
            sil = np.nan

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        print(f"[{idx}/{total}] UMAP(nn={nn}, md={md}, nc={nc}) + "
              f"HDBSCAN(mcs={mcs}, ms={ms}, eps={eps}) "
              f"=> Clusters={n_clusters}, Sil={sil:.4f}, Noise={noise_ratio:.3f}")

        rows.append({
            "nn": nn, "md": md, "nc": nc, "umap_metric": met_r,
            "mcs": mcs, "ms": ms, "eps": eps, "hdb_metric": met_c,
            "Clusters": n_clusters,
            "Silhouette": sil,
            "NoiseRatio": noise_ratio,
            "Score": (sil if not np.isnan(sil) else -1) - noise_ratio,
            "TimeSec": round(time() - t0, 3)
        })

# ================== 4. 保存 & 查看结果 ==================
df = pd.DataFrame(rows)
df_sorted = df.sort_values(by="Score", ascending=False)

print("\nTop 5 参数组合：")
print(df_sorted.head(5))

# 额外：保存到 CSV 文件，防止结果丢失
df_sorted.to_csv("umap_hdbscan_results_fashion.csv", index=False)
print("\n完整结果已保存到 umap_hdbscan_results.csv")


数据形状: (4722, 64)


/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/root/miniconda3/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


[1/432] UMAP(nn=10, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=5, eps=0.0) => Clusters=78, Sil=0.6719, Noise=0.144
[2/432] UMAP(nn=10, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=5, eps=0.1) => Clusters=77, Sil=0.6747, Noise=0.144
[3/432] UMAP(nn=10, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=10, eps=0.0) => Clusters=68, Sil=0.7059, Noise=0.171
[4/432] UMAP(nn=10, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=10, eps=0.1) => Clusters=68, Sil=0.7059, Noise=0.171
[5/432] UMAP(nn=10, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=20, eps=0.0) => Clusters=58, Sil=0.7336, Noise=0.218
[6/432] UMAP(nn=10, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=20, eps=0.1) => Clusters=58, Sil=0.7336, Noise=0.218
[7/432] UMAP(nn=10, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=50, eps=0.0) => Clusters=29, Sil=0.7160, Noise=0.265
[8/432] UMAP(nn=10, md=0.0, nc=4) + HDBSCAN(mcs=20, ms=50, eps=0.1) => Clusters=29, Sil=0.7160, Noise=0.265
[9/432] UMAP(nn=10, md=0.0, nc=4) + HDBSCAN(mcs=50, ms=5, eps=0.0) => Clusters=33, Sil=0.6317, Noise=0.146
[10/432] UMAP(nn=10, md=0.0, nc

/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[25/432] UMAP(nn=10, md=0.0, nc=8) + HDBSCAN(mcs=20, ms=5, eps=0.0) => Clusters=72, Sil=0.6714, Noise=0.116
[26/432] UMAP(nn=10, md=0.0, nc=8) + HDBSCAN(mcs=20, ms=5, eps=0.1) => Clusters=72, Sil=0.6714, Noise=0.116
[27/432] UMAP(nn=10, md=0.0, nc=8) + HDBSCAN(mcs=20, ms=10, eps=0.0) => Clusters=68, Sil=0.6894, Noise=0.149
[28/432] UMAP(nn=10, md=0.0, nc=8) + HDBSCAN(mcs=20, ms=10, eps=0.1) => Clusters=68, Sil=0.6894, Noise=0.149
[29/432] UMAP(nn=10, md=0.0, nc=8) + HDBSCAN(mcs=20, ms=20, eps=0.0) => Clusters=56, Sil=0.7070, Noise=0.182
[30/432] UMAP(nn=10, md=0.0, nc=8) + HDBSCAN(mcs=20, ms=20, eps=0.1) => Clusters=56, Sil=0.7070, Noise=0.182
[31/432] UMAP(nn=10, md=0.0, nc=8) + HDBSCAN(mcs=20, ms=50, eps=0.0) => Clusters=28, Sil=0.7200, Noise=0.289
[32/432] UMAP(nn=10, md=0.0, nc=8) + HDBSCAN(mcs=20, ms=50, eps=0.1) => Clusters=28, Sil=0.7200, Noise=0.289
[33/432] UMAP(nn=10, md=0.0, nc=8) + HDBSCAN(mcs=50, ms=5, eps=0.0) => Clusters=34, Sil=0.6318, Noise=0.155
[34/432] UMAP(nn=10, m

...

[361/432] UMAP(nn=50, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=5, eps=0.0) => Clusters=55, Sil=0.6197, Noise=0.233
[362/432] UMAP(nn=50, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=5, eps=0.1) => Clusters=55, Sil=0.6197, Noise=0.233
[363/432] UMAP(nn=50, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=10, eps=0.0) => Clusters=52, Sil=0.6268, Noise=0.246
[364/432] UMAP(nn=50, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=10, eps=0.1) => Clusters=52, Sil=0.6268, Noise=0.246
[365/432] UMAP(nn=50, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=20, eps=0.0) => Clusters=48, Sil=0.6684, Noise=0.305
[366/432] UMAP(nn=50, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=20, eps=0.1) => Clusters=48, Sil=0.6684, Noise=0.305
[367/432] UMAP(nn=50, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=50, eps=0.0) => Clusters=2, Sil=0.7051, Noise=0.007
[368/432] UMAP(nn=50, md=0.1, nc=4) + HDBSCAN(mcs=20, ms=50, eps=0.1) => Clusters=2, Sil=0.7051, Noise=0.007
[369/432] UMAP(nn=50, md=0.1, nc=4) + HDBSCAN(mcs=50, ms=5, eps=0.0) => Clusters=32, Sil=0.5606, Noise=0.241
[370/432] UMAP(

/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[385/432] UMAP(nn=50, md=0.1, nc=8) + HDBSCAN(mcs=20, ms=5, eps=0.0) => Clusters=58, Sil=0.5769, Noise=0.206
[386/432] UMAP(nn=50, md=0.1, nc=8) + HDBSCAN(mcs=20, ms=5, eps=0.1) => Clusters=58, Sil=0.5769, Noise=0.206
[387/432] UMAP(nn=50, md=0.1, nc=8) + HDBSCAN(mcs=20, ms=10, eps=0.0) => Clusters=52, Sil=0.6156, Noise=0.252
[388/432] UMAP(nn=50, md=0.1, nc=8) + HDBSCAN(mcs=20, ms=10, eps=0.1) => Clusters=52, Sil=0.6156, Noise=0.252
[389/432] UMAP(nn=50, md=0.1, nc=8) + HDBSCAN(mcs=20, ms=20, eps=0.0) => Clusters=49, Sil=0.6404, Noise=0.288
[390/432] UMAP(nn=50, md=0.1, nc=8) + HDBSCAN(mcs=20, ms=20, eps=0.1) => Clusters=49, Sil=0.6404, Noise=0.288
[391/432] UMAP(nn=50, md=0.1, nc=8) + HDBSCAN(mcs=20, ms=50, eps=0.0) => Clusters=3, Sil=0.3813, Noise=0.007
[392/432] UMAP(nn=50, md=0.1, nc=8) + HDBSCAN(mcs=20, ms=50, eps=0.1) => Clusters=3, Sil=0.3813, Noise=0.007
[393/432] UMAP(nn=50, md=0.1, nc=8) + HDBSCAN(mcs=50, ms=5, eps=0.0) => Clusters=28, Sil=0.5252, Noise=0.184
[394/432] UMAP(

/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[409/432] UMAP(nn=50, md=0.1, nc=16) + HDBSCAN(mcs=20, ms=5, eps=0.0) => Clusters=55, Sil=0.5872, Noise=0.232
[410/432] UMAP(nn=50, md=0.1, nc=16) + HDBSCAN(mcs=20, ms=5, eps=0.1) => Clusters=55, Sil=0.5872, Noise=0.232
[411/432] UMAP(nn=50, md=0.1, nc=16) + HDBSCAN(mcs=20, ms=10, eps=0.0) => Clusters=51, Sil=0.6234, Noise=0.268
[412/432] UMAP(nn=50, md=0.1, nc=16) + HDBSCAN(mcs=20, ms=10, eps=0.1) => Clusters=51, Sil=0.6234, Noise=0.268
[413/432] UMAP(nn=50, md=0.1, nc=16) + HDBSCAN(mcs=20, ms=20, eps=0.0) => Clusters=44, Sil=0.6262, Noise=0.272
[414/432] UMAP(nn=50, md=0.1, nc=16) + HDBSCAN(mcs=20, ms=20, eps=0.1) => Clusters=44, Sil=0.6262, Noise=0.272
[415/432] UMAP(nn=50, md=0.1, nc=16) + HDBSCAN(mcs=20, ms=50, eps=0.0) => Clusters=3, Sil=0.3781, Noise=0.007
[416/432] UMAP(nn=50, md=0.1, nc=16) + HDBSCAN(mcs=20, ms=50, eps=0.1) => Clusters=3, Sil=0.3781, Noise=0.007
[417/432] UMAP(nn=50, md=0.1, nc=16) + HDBSCAN(mcs=50, ms=5, eps=0.0) => Clusters=32, Sil=0.5554, Noise=0.223
[418/4

In [6]:
import pandas as pd

# 读取结果
df = pd.read_csv("umap_hdbscan_results_fashion.csv")

# 筛选条件：簇数适中 + 噪声不过大
df_filtered = df[
    (df["Clusters"] >= 10) & 
    (df["Clusters"] <= 30) & 
    (df["NoiseRatio"] <= 0.3)
]

# 按 Score 排序，取 Top 10
df_top = df_filtered.sort_values(by="Score", ascending=False).head(10)

print("===== 符合条件的 Top 10 参数组合 =====")
print(df_top)

# 保存筛选结果
df_top.to_csv("umap_hdbscan_filtered_fashion.csv", index=False)
print("\n筛选后的结果已保存到 umap_hdbscan_filtered_fashion.csv")

===== 符合条件的 Top 10 参数组合 =====
    nn   md  nc umap_metric  mcs  ms  eps hdb_metric  Clusters  Silhouette  \
55  20  0.0   8   euclidean   50  10  0.0  euclidean        27    0.594937   
54  20  0.0   8   euclidean   50  10  0.1  euclidean        27    0.594937   
59  10  0.0  16   euclidean   50  20  0.1  euclidean        30    0.686801   
58  10  0.0  16   euclidean   50  20  0.0  euclidean        30    0.686801   
67  20  0.0   8   euclidean   50  20  0.1  euclidean        29    0.633730   
66  20  0.0   8   euclidean   50  20  0.0  euclidean        29    0.633730   
69  10  0.0   4   euclidean   20  50  0.1  euclidean        29    0.716013   
68  10  0.0   4   euclidean   20  50  0.0  euclidean        29    0.716013   
70  20  0.0  16   euclidean   50  20  0.1  euclidean        28    0.619458   
71  20  0.0  16   euclidean   50  20  0.0  euclidean        28    0.619458   

    NoiseRatio     Score  TimeSec  
55    0.129818  0.465119    0.390  
54    0.129818  0.465119    0.386  
59 

1. 基于 Fashion 最优参数重新生成聚类

In [6]:
# # 1. Fashion最优参数（示例，需从搜索结果中选Top1）
# best_umap_params = {"n_neighbors":20, "min_dist":0.0, "n_components":8, "metric":"euclidean"}
# best_hdb_params = {"min_cluster_size":50, "min_samples":10, "cluster_selection_epsilon":0.0, "metric":"euclidean"}

# # 1. Fashion最优参数（示例，需从搜索结果中选Top1）
# best_umap_params = {"n_neighbors":10, "min_dist":0.0, "n_components":4, "metric":"euclidean"}
# best_hdb_params = {"min_cluster_size":20, "min_samples":20, "cluster_selection_epsilon":0.1, "metric":"euclidean"}

# # 1. Fashion最优参数（示例，需从搜索结果中选Top1）
# best_umap_params = {"n_neighbors":10, "min_dist":0.1, "n_components":8, "metric":"euclidean"}
# best_hdb_params = {"min_cluster_size":20, "min_samples":20, "cluster_selection_epsilon":0.0, "metric":"euclidean"}

# 新增 @20 和细粒度搜索后
# nn   md  nc  mcs  ms   eps  Clusters  Silhouette  NoiseRatio     Score
#  8  0.0   4   20  25  0.00        51    0.747387    0.259848  0.487540
best_umap_params = {"n_neighbors":8, "min_dist":0.0, "n_components":4, "metric":"euclidean"}
best_hdb_params = {"min_cluster_size":20, "min_samples":25, "cluster_selection_epsilon":0.05, "metric":"euclidean"}

# 2. 加载Fashion的PCA64嵌入（路径修改）
X_pca = pickle.load(open("../data/fashion/handled/pca64_itm_emb_np.pkl", "rb"))
X_scaled = StandardScaler().fit_transform(X_pca)

# 3. UMAP降维（适配Fashion的细粒度语义）
best_reducer = umap.UMAP(**best_umap_params, random_state=42)
Z_best = best_reducer.fit_transform(X_scaled)

# 4. HDBSCAN聚类（适配低物品基数）
best_clusterer = hdbscan.HDBSCAN(**best_hdb_params)
labels_best = best_clusterer.fit_predict(Z_best)
probs_best = best_clusterer.probabilities_

# 5. 验证Fashion聚类结果（重点关注长尾噪声）
noise_ratio = np.mean(labels_best == -1)
print(f"Fashion聚类：簇数量={len(set(labels_best))-(1 if -1 in labels_best else 0)}, "
      f"噪声比例={noise_ratio:.3f}（需≤0.25）")

/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Fashion聚类：簇数量=48, 噪声比例=0.229（需≤0.25）


2. 保存文件到 Fashion 的handled目录（路径修改）

In [7]:
# ================== 1. （已完成）用最优参数生成聚类结果 ==================
# 加载Fashion PCA64嵌入
X_pca = pickle.load(open("../data/fashion/handled/pca64_itm_emb_np.pkl", "rb"))
X_scaled = StandardScaler().fit_transform(X_pca)

# nn   md  nc  mcs  ms   eps  Clusters  Silhouette  NoiseRatio     Score
#  8  0.0   4   25  25  0.00        50    0.742579    0.251377  0.491202

# 最优UMAP降维（你已确定的参数）
best_umap_params = {"n_neighbors":8, "min_dist":0.0, "n_components":4, "metric":"euclidean"}
best_reducer = umap.UMAP(**best_umap_params, random_state=42)
Z_best = best_reducer.fit_transform(X_scaled)

# 最优HDBSCAN聚类（你已确定的参数）
best_hdb_params = {"min_cluster_size":20, "min_samples":25, "cluster_selection_epsilon":0.05, "metric":"euclidean"}
best_clusterer = hdbscan.HDBSCAN(**best_hdb_params)
labels_best = best_clusterer.fit_predict(Z_best)  # 簇标签（含噪声-1）
probs_best = best_clusterer.probabilities_        # 核心点隶属度


# ================== 2. （新增！）计算加权簇中心 cluster_centers_final ==================
# 步骤1：筛选有效簇（排除噪声标签-1）
valid_cluster_ids = np.unique(labels_best[labels_best != -1])  # 所有非噪声的簇ID
cluster_centers_final = []  # 存储最终的加权簇中心

# 步骤2：对每个有效簇，按核心隶属度加权计算中心（文档中“语义锚点需保留核心点置信度”）
for cid in valid_cluster_ids:
    # 筛选该簇的所有物品：嵌入（X_pca）、核心隶属度（probs_best）
    cluster_mask = labels_best == cid  # 该簇的物品掩码
    cluster_embeddings = X_pca[cluster_mask]  # 该簇的物品嵌入（用PCA64嵌入，与文档一致）
    cluster_probs = probs_best[cluster_mask]  # 该簇物品的核心隶属度（权重）
    
    # 加权平均计算簇中心（隶属度越高的物品，对中心贡献越大）
    weighted_center = np.average(cluster_embeddings, axis=0, weights=cluster_probs)
    cluster_centers_final.append(weighted_center)

# 转为numpy数组（便于后续模糊隶属度计算和保存）
cluster_centers_final = np.array(cluster_centers_final)


# ================== 3. （新增！）计算模糊隶属度向量 fuzzy_U_final ==================
# （若之前未计算，需补充这一步，否则会报同样的NameError）
def compute_fuzzy_membership(item_emb, cluster_labels, cluster_probs, cluster_centers, fuzzy_m=2):
    N = len(item_emb)
    C = len(cluster_centers)
    fuzzy_U = np.zeros((N, C))  # 每个物品的模糊隶属度向量（N个物品，C个簇）
    
    # 处理非噪声点（保留主簇语义+次要簇关联）
    non_noise_mask = cluster_labels != -1
    valid_cluster_ids = np.unique(cluster_labels[non_noise_mask])
    for i in np.where(non_noise_mask)[0]:
        cid = cluster_labels[i]
        cid_idx = np.where(valid_cluster_ids == cid)[0][0]  # 簇ID映射到索引
        main_prob = cluster_probs[i]
        
        # 计算到各簇中心的距离（距离越近，隶属度越高）
        dists = np.linalg.norm(item_emb[i] - cluster_centers, axis=1)
        dists[dists < 1e-8] = 1e-8  # 避免除零
        inv_dists = 1 / dists
        
        # 分配隶属度并归一化
        fuzzy_U[i, cid_idx] = main_prob * inv_dists[cid_idx]
        for k in range(C):
            if k != cid_idx:
                fuzzy_U[i, k] = (1 - main_prob) * inv_dists[k]
        fuzzy_U[i] /= np.sum(fuzzy_U[i])
    
    # 处理噪声点（极度模糊，按距离分配）
    noise_mask = cluster_labels == -1
    for i in np.where(noise_mask)[0]:
        dists = np.linalg.norm(item_emb[i] - cluster_centers, axis=1)
        dists[dists < 1e-8] = 1e-8
        inv_dists = 1 / dists
        fuzzy_U[i] = inv_dists ** (2 / (fuzzy_m - 1))
        fuzzy_U[i] /= np.sum(fuzzy_U[i])
    
    return fuzzy_U

# 调用函数计算模糊隶属度（用PCA64嵌入，与文档双视图建模一致）
fuzzy_U_final = compute_fuzzy_membership(
    item_emb=X_pca,
    cluster_labels=labels_best,
    cluster_probs=probs_best,
    cluster_centers=cluster_centers_final,
    fuzzy_m=2  # 模糊指数，与文档一致
)


# ================== 4. （已写）保存文件（现在变量均已定义，无报错） ==================
save_dir = "../data/fashion/handled/"
pickle.dump(labels_best, open(f"{save_dir}/hdbscan_best_labels.pkl", "wb"))
pickle.dump(cluster_centers_final, open(f"{save_dir}/hdbscan_cluster_centers.pkl", "wb"))  # 已定义
pickle.dump(fuzzy_U_final, open(f"{save_dir}/hdbscan_fuzzy_U.pkl", "wb"))  # 已定义
pickle.dump(probs_best, open(f"{save_dir}/hdbscan_core_probs.pkl", "wb"))

print("所有文件保存成功！")

/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


所有文件保存成功！


In [8]:
# 验证变量是否存在且格式正确
print(f"cluster_centers_final 形状: {cluster_centers_final.shape}")  # 预期 (27, 64)（27个簇，64维嵌入）
print(f"fuzzy_U_final 形状: {fuzzy_U_final.shape}")                  # 预期 (4722, 27)（4722个物品，27个簇）
print(f"labels_best 形状: {labels_best.shape}")                      # 预期 (4722,)

cluster_centers_final 形状: (47, 64)
fuzzy_U_final 形状: (4722, 47)
labels_best 形状: (4722,)
